In [1]:
#step 1 import image
%matplotlib inline
import torchvision.datasets
import math
import torchvision.transforms as tvt
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import wget
import zipfile
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.datasets as datasets
import torchvision.models as models
import torchvision.transforms as tfms
from torch.utils.data import DataLoader, Subset, Dataset
from torchvision.utils import make_grid
from PIL import Image
from time import time
from tqdm import tqdm
from transformers import ViTConfig, ViTModel


device = torch.device('cuda:0')

image_size = 64
batch_size = 1024
dataset = torchvision.datasets.CelebA("../../celeba/datasets/",split='train', transform=tvt.Compose([
                                  tvt.Resize((image_size,image_size)),
                                  tvt.ToTensor(),
                                  tvt.Normalize(mean=[0.5, 0.5, 0.5],
                                                std=[0.5, 0.5, 0.5])
                              ]))

test_dataset = torchvision.datasets.CelebA("../../celeba/datasets/",split='test', transform=tvt.Compose([
                                  tvt.Resize((image_size,image_size)),
                                  tvt.ToTensor(),
                                  tvt.Normalize(mean=[0.5, 0.5, 0.5],
                                                std=[0.5, 0.5, 0.5])
                              ]))

training_data_loader = torch.utils.data.DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=True, drop_last=True)
print('Done')

/home/shenyu/miniconda3/envs/DLcourse/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Done


In [2]:
class VisionTransformer(nn.Module):
    def __init__(self, vit):
        super(VisionTransformer, self).__init__()
        self.vit = vit
        self.seq = nn.Sequential(
            nn.Linear(768, 256),
            nn.ReLU(),
            nn.Linear(256, 1)
        )
    
    def forward(self, x):
        z = self.vit(x)
        m = z.last_hidden_state
        g = m[:,0]
        y = self.seq(g)
        return y 

In [3]:
import random
import numpy as np
import torch
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from sklearn.metrics import accuracy_score, precision_score
from sklearn.metrics import confusion_matrix
from collections import OrderedDict
import seaborn as sns


class DRO_loss(torch.nn.Module):
    def __init__(self, eta, k):
        super(DRO_loss, self).__init__()
        self.eta = eta
        self.k = k
        self.logsig = torch.nn.LogSigmoid()
        self.relu = torch.nn.ReLU()
    
    def forward(self, x, y):
        bce = -1*y*self.logsig(x) - (1-y)*self.logsig(-x)
        if self.k > 0:
            bce = self.relu(bce - self.eta)
            bce = bce**self.k
            return bce.mean()
        else:
            return bce.mean()


def seed_everything(seed):
    """
    Changes the seed for reproducibility. 
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

def train_model():
    epoch = 55
    configuration = ViTConfig(num_hidden_layers = 8, num_attention_heads = 8, 
                          intermediate_size = 768, image_size= 64, patch_size = 16)
    vit = ViTModel(configuration)
    configuration = vit.config
    vit = vit.to(device)
    model = VisionTransformer(vit)
    model = model.to(device)
    
    criterion = DRO_loss(eta=0.25, k=1.5)
    optimizer = optim.AdamW(model.parameters(), lr=1e-4)

    #fair_optimizer = optim.Adam(fairnet.parameters(), lr = 1e-3)
    #fair_epoch = 30

    for epoches in range(epoch):
        with tqdm(training_data_loader, unit="batch") as tepoch:
            for train_input, attributes in tepoch:
                # Transfer data to GPU if possible. 
                train_input = train_input.to(device)
                sensitive, train_target = attributes[:,20], attributes[:,2]
               
                train_target = train_target.float().to(device)
                train_target = train_target.unsqueeze(1)
                optimizer.zero_grad()

                # Learner update step.
                outputs = model(train_input)
                loss = criterion(outputs, train_target)
                loss.backward(retain_graph=True)
                #logger_learner.add_values(logging_dict)
                optimizer.step()
                tepoch.set_description(f"epoch %2f " % epoches)
                tepoch.set_postfix(ut_loss = loss.item())
        # Reset the dataloader if out of data.
        #model.load_state_dict(torch.load(PATH), False)
        test_pred = []
        test_gt = []
        sense_gt = []
        female_predic = []
        female_gt = []
        male_predic = []
        male_gt = []



    # Evaluate on test set.
        for step, (test_input, attributes) in enumerate(test_data_loader):
            sensitive, test_target = attributes[:,20], attributes[:,2]
            test_input = test_input.to(device)
            test_target = test_target.to(device)

            gt = test_target.detach().cpu().numpy()
            sen = sensitive.detach().cpu().numpy()
            test_gt.extend(gt)
            sense_gt.extend(sen)

            # Todo: split according to sensitive attribute
            # Todo: combine all batch togather

            with torch.no_grad():
                test_pred_= model(test_input)
                test_pred_ = torch.sigmoid(test_pred_)
                test_pred.extend(torch.round(test_pred_.squeeze(1)).detach().cpu().numpy())

        for i in range(len(sense_gt)):
            if sense_gt[i] == 0:
                female_predic.append(test_pred[i])
                female_gt.append(test_gt[i])
            else:
                male_predic.append(test_pred[i])
                male_gt.append(test_gt[i])
        female_CM = confusion_matrix(female_gt, female_predic)    
        male_CM = confusion_matrix(male_gt, male_predic) 
        female_dp = (female_CM[1][1]+female_CM[0][1])/(female_CM[0][0]+female_CM[0][1]+female_CM[1][0]+female_CM[1][1])
        male_dp = (male_CM[1][1]+male_CM[0][1])/(male_CM[0][0]+male_CM[0][1]+male_CM[1][0]+male_CM[1][1])
        female_TPR = female_CM[1][1]/(female_CM[1][1]+female_CM[1][0])
        male_TPR = male_CM[1][1]/(male_CM[1][1]+male_CM[1][0])
        female_FPR = female_CM[0][1]/(female_CM[0][1]+female_CM[0][0])
        male_FPR = male_CM[0][1]/(male_CM[0][1]+male_CM[0][0])

        print('Female TPR', female_TPR)
        print('male TPR', male_TPR)
        print('DP',abs(female_dp - male_dp))
        print('EOP', abs(female_TPR - male_TPR))
        print('EoD',0.5*(abs(female_FPR-male_FPR)+ abs(female_TPR-male_TPR)))
        print('acc', accuracy_score(test_gt, test_pred))
        print('Trade off',accuracy_score(test_gt, test_pred)*(1-0.5*(abs(female_FPR-male_FPR)+ abs(female_TPR-male_TPR))) )



seed_everything(4096)    
train_model()


epoch 0.000000 : 100%|██████| 158/158 [02:30<00:00,  1.05batch/s, ut_loss=0.216]


Female TPR 0.8577652485904664
male TPR 0.46802794196668457
DP 0.5316184231069476
EOP 0.38973730662378187
EoD 0.38335848260766037
acc 0.758069490131579
Trade off 0.4674571206835741


epoch 1.000000 : 100%|██████| 158/158 [02:29<00:00,  1.06batch/s, ut_loss=0.177]


Female TPR 0.9388123148267422
male TPR 0.7254691689008043
DP 0.5079374952706557
EOP 0.21334314592593795
EoD 0.32870556824818303
acc 0.7560649671052632
Trade off 0.5075422024603838


epoch 2.000000 : 100%|██████| 158/158 [02:29<00:00,  1.06batch/s, ut_loss=0.176]


...

Female TPR 0.8469741744828473
male TPR 0.5657541599570585
DP 0.4567801323370031
EOP 0.2812200145257888
EoD 0.2906537576228341
acc 0.7586348684210527
Trade off 0.5381347932507694


epoch 53.000000 : 100%|████| 158/158 [02:33<00:00,  1.03batch/s, ut_loss=0.0138]


Female TPR 0.819138019497178
male TPR 0.5705850778314546
DP 0.4223056694554209
EOP 0.2485529416657234
EoD 0.2533173806196834
acc 0.7567845394736842
Trade off 0.5650778622407372


epoch 54.000000 : 100%|████| 158/158 [02:33<00:00,  1.03batch/s, ut_loss=0.0107]


Female TPR 0.8565362840967576
male TPR 0.5939167556029883
DP 0.4489990799911139
EOP 0.26261952849376935
EoD 0.2846418008618217
acc 0.7529296875
Trade off 0.5386144253276713
